# HDI Registration

In [1]:
# Extract cores from TMA using morphological operations
# import custom modules
from miaaim.reg.elastix import Elastix
from miaaim.reg._utils import FormatFijiLandmarkPoints
from miaaim.io.imread import _import

# import other modules
import pandas as pd
from pathlib import Path
import os
# check versions and environment executable
import sys

sys.version
sys.executable

ModuleNotFoundError: No module named 'miaaim'

In [2]:
def RegisterDataSet(core_name, data_folder):
    """Helper function to perform MSI to IMC registration
    with H&E intermediate.

    core_name: string indicating core name (in data folder)

    data_folder: Pathlib object indicating data folder for project
    """

    # set path to core folder (root_folder below)
    core_folder = data_folder.joinpath(f"{core_name}")
    # set the path to the imaging data
    he = data_folder.joinpath(f"{core_name}/preprocessing/he/{core_name}_core.nii")
    msi = data_folder.joinpath(f"{core_name}/preprocessing/msi/{core_name}_core_msi_UMAP.nii")
    imc = data_folder.joinpath(f"{core_name}/preprocessing/imc/{core_name}_core_UMAP.nii")

    # set path to original MSI data to transform
    input_msi = core_folder.joinpath(f"input/msi/{core_name}.hdf5")

    # set output directory
    registration_dir = data_folder.joinpath(f"{core_name}/registration")

    # create landmark text files
    he_landmarks = FormatFijiLandmarkPoints(registration_dir.joinpath(f"{core_name}_core.txt"),
                                            selection_type='index')
    msi_landmarks = FormatFijiLandmarkPoints(registration_dir.joinpath(f"{core_name}_core_msi_UMAP.txt"),
                                             selection_type='index')
    imc_landmarks = FormatFijiLandmarkPoints(registration_dir.joinpath(f"{core_name}_core_UMAP.txt"),
                                             selection_type='index')

    # set parameter files
    landmark_p_msi_he = [registration_dir.joinpath("10_landmark_affine.txt"),
                         registration_dir.joinpath("11_landmark_bspline.txt")]
    p_msi_he = [registration_dir.joinpath("12_aMI_affine.txt"),
                registration_dir.joinpath("13_aMI_bspline.txt")]
    landmark_p_he_imc = [registration_dir.joinpath("20_landmark_affine.txt"),
                         registration_dir.joinpath("21_landmark_bspline.txt")]
    p_he_imc = [registration_dir.joinpath("22_aMI_affine.txt"),
                registration_dir.joinpath("23_aMI_bspline.txt")]

    # create elastix registration class
    msi_he_reg = Elastix(root_folder=core_folder,
                         name="msi-he",
                         resume=False)

    # perform registration procedure
    msi_he_reg.Register(fixed=he,
                        moving=msi,
                        p=p_msi_he,
                        landmark_p=None,
                        out_dir=None,
                        fp=he_landmarks,
                        mp=msi_landmarks,
                        fMask=None,
                        multichannel=True)

    # transform UMAP msi representation for visualization
    msi_he_reg.Transform(in_im=None,
                         out_dir=None,
                         tps=None,
                         target_size=None,
                         pad=None,
                         trim=None,
                         crops=None,
                         out_ext=".nii")

    # export QC information
    msi_he_reg.QC()

In [3]:
# set path to data folder
data_folder = Path("/opt/miaaim-20220912-TMA4/data")
# get list of all core names in the data folder
# check to make sure we have the core folders for only prostate samples which have
# multimodal data (here, indicated by /registration folder)
core_folders = [data_folder.joinpath(c) for c in os.listdir(data_folder) if data_folder.joinpath(c).is_dir()]
core_folders = [c for c in core_folders if Path(f"{c}/registration").is_dir()]

# iterate through data set and register
for c in core_folders[1:2]:
    # register
    RegisterDataSet(core_name=c.name, data_folder=data_folder)

NameError: name 'Path' is not defined